---
title: "Untitled"
format: html
---

In [23]:
# Setup all functions to run at once and return all the results to user

####################
# Import Packages
####################
import pandas as pd
import sqlite3
import math
import numpy as np
import sys
import os
from tabulate import tabulate

####################
# Import Functions
####################
from ration_balancer_functions import read_input, fl_get_rows, get_nutrient_intakes

# Modify path temporarily to import functions from NASEM_functions directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from NASEM_functions.ration_balancing_equations import calculate_Dt_DMIn_Lact1, calculate_Du_MiCP_g, AA_calculations, calculate_An_DEIn, calculate_Mlk_NP_g, calculate_An_NE, calculate_Mlk_Fat_g, calculate_Mlk_Prod_comp, calculate_Mlk_Prod_MPalow, calculate_Mlk_Prod_NEalow 
from NASEM_functions.ME_functions import calculate_ME_requirement, calculate_An_MEmUse, calculate_An_MEgain, calculate_Gest_MEuse, calculate_Trg_Mlk_MEout
from NASEM_functions.MP_functions import calculate_MP_requirement, calculate_An_MPm_g_Trg, calculate_Body_MPuse_g_Trg, calculate_Gest_MPuse_g_Trg, calculate_Mlk_MPuse_g_Trg


# Deleting to clean up environment
del(current_dir)
del(parent_dir)

In [24]:
def NASEM_model():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    diet_info, animal_input, equation_selection = read_input('input.txt')
    
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = diet_info['Feedstuff'].tolist()
    feed_data = fl_get_rows(list_of_feeds)

    # Scale %_DM_intake to 100%
    user_perc = diet_info['%_DM_user'].sum()
    scaling_factor = 100 / user_perc

    # Should be called 'Fd_DMInp' instead of %_DM_intake
    diet_info['%_DM_intake'] = diet_info['%_DM_user'] * scaling_factor
    # Adjust so sum is exactly 100
    adjustment = 100 - diet_info['%_DM_intake'].sum()
    diet_info['%_DM_intake'] += adjustment / len(diet_info)

    # Predict DMI
    if equation_selection['DMI_pred'] == 0:
        animal_input['DMI'] = calculate_Dt_DMIn_Lact1(animal_input['An_Parity_rl'], animal_input['Trg_MilkProd'], animal_input['An_BW'], animal_input['An_BCS'],
                                                      animal_input['An_LactDay'], animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'])
    # Should be called 'Fd_DMIn' for consistency with R code
    diet_info['kg_intake'] = diet_info['%_DM_intake'] / 100 * animal_input['DMI']

########################################
# Step 2: Feed Based Calculations
########################################
    diet_info = get_nutrient_intakes(diet_info, feed_data, animal_input, equation_selection)

########################################
# Step 3: Microbial Protein Calculations
########################################
    Du_MiN_NRC2021_g = calculate_Du_MiCP_g(diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], diet_info.loc['Diet', 'Fd_St_kg/d'], diet_info.loc['Diet', 'Fd_CP_kg/d'], diet_info.loc['Diet', 'Fd_ADF_kg/d'], 
                                    diet_info.loc['Diet', 'Fd_ForWetIn'], diet_info.loc['Diet', 'Fd_RUPIn'], diet_info.loc['Diet', 'Fd_ForNDFIn'], diet_info.loc['Diet', 'Dt_RDPIn'])

########################################
# Step 4: Amino Acid Calculations
########################################
    AA_values = AA_calculations(Du_MiN_NRC2021_g, feed_data, diet_info)

########################################
# Step 5: Other Calculations
########################################
# Intake calculations that require additional steps, need results from other calculations or values that need to be calculated for other functions

    # This function could be renamed as it is doing all the DE intake calculations
    An_DEIn, An_DENPNCPIn, An_DETPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, Fe_CP, Fe_CPend_g = calculate_An_DEIn(diet_info.loc['Diet', 'Fd_DigNDFIn_Base'], diet_info.loc['Diet', 'Fd_NDFIn'], 
                                                                                                                    diet_info.loc['Diet', 'Fd_DigStIn_Base'], diet_info.loc['Diet', 'Fd_St_kg/d'], 
                                                                                                                    diet_info.loc['Diet','Fd_DigrOMtIn'], diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_RUPIn'],
                                                                                                                    diet_info.loc['Diet', 'Fd_idRUPIn'], diet_info.loc['Diet', 'Fd_NPNCPIn'], diet_info.loc['Diet', 'Fd_DigFAIn'], 
                                                                                                                    Du_MiN_NRC2021_g, animal_input['An_BW'], animal_input['DMI'])

    # Predicted milk protein
    Mlk_NP_g, Du_idMiCP_g = calculate_Mlk_NP_g(AA_values, diet_info.loc['Diet', 'Fd_idRUPIn'], Du_MiN_NRC2021_g, An_DEIn, An_DETPIn, An_DENPNCPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, animal_input['An_BW'], animal_input['DMI'])

    # Net energy/Metabolizable energy
    An_NE, An_MEIn = calculate_An_NE(diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_FAIn'], Mlk_NP_g, An_DEIn, An_DigNDFIn, Fe_CP, Fe_CPend_g, animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], 
                            animal_input['Trg_FrmGain'], animal_input['An_GestDay'], animal_input['An_GestLength'], animal_input['An_LactDay'], animal_input['Trg_RsrvGain'], animal_input['Fet_BWbrth'],
                            animal_input['An_AgeDay'], animal_input['An_Parity_rl'])
 
########################################
# Step 6: Requirement Calculations
########################################

    # Metabolizable Energy Requirements
    Trg_MEuse, An_MEmUse, An_MEgain, Gest_MEuse, Trg_Mlk_MEout = calculate_ME_requirement(animal_input['An_BW'], animal_input['DMI'], animal_input['Trg_MilkProd'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'], animal_input['An_GestDay'], 
                                                                                          animal_input['An_GestLength'], animal_input['An_AgeDay'], animal_input['Fet_BWbrth'], animal_input['An_LactDay'], animal_input['An_Parity_rl'],
                                                                                          animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'], animal_input['Trg_RsrvGain'])
    # Metabolizable Protein Requirements
    An_MPuse_g_Trg, An_MPm_g_Trg, Body_MPUse_g_Trg, Gest_MPUse_g_Trg, Mlk_MPUse_g_Trg = calculate_MP_requirement(diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'],
                                                                                                                 animal_input['Trg_RsrvGain'], animal_input['An_GestDay'], animal_input['An_GestLength'], animal_input['An_AgeDay'], animal_input['Fet_BWbrth'],
                                                                                                                 animal_input['An_LactDay'], animal_input['An_Parity_rl'], animal_input['Trg_MilkProd'], animal_input['Trg_MilkTPp'])    

########################################
# Step 7: Performance Calculations
########################################

    # Predicted milk fat
    Mlk_Fat_g = calculate_Mlk_Fat_g(AA_values, diet_info.loc['Diet', 'Fd_FAIn'], diet_info.loc['Diet', 'Fd_DigC160In'], diet_info.loc['Diet', 'Fd_DigC183In'], animal_input['An_LactDay'], animal_input['DMI'])

    # Predicted milk yield
    Mlk_Prod_comp = calculate_Mlk_Prod_comp(Mlk_NP_g, Mlk_Fat_g, An_DEIn, animal_input['An_LactDay'], animal_input['An_Parity_rl']) 
  
    # MP Allowable Milk
    Mlk_Prod_MPalow = calculate_Mlk_Prod_MPalow(An_MPuse_g_Trg, Mlk_MPUse_g_Trg, diet_info.loc['Diet', 'Fd_idRUPIn'], Du_idMiCP_g, animal_input['Trg_MilkTPp'])

    # NE Allowable Milk
    Mlk_Prod_NEalow = calculate_Mlk_Prod_NEalow(An_MEIn, An_MEgain, An_MEmUse, Gest_MEuse, animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'])

########################################
# Step 8: *Temporary* Add values of interest to list
########################################
    # Milk Fat %
    milk_fat = (Mlk_Fat_g / 1000) / Mlk_Prod_comp *100
    # Milk Protein %
    milk_protein = (Mlk_NP_g / 1000) / Mlk_Prod_comp *100

    model_results = {
    'milk_fat': milk_fat,
    'milk_protein': milk_protein,
    'Mlk_Prod_comp': Mlk_Prod_comp,
    'Mlk_Prod_MPalow': Mlk_Prod_MPalow,
    'Mlk_Prod_NEalow': Mlk_Prod_NEalow,
    'Trg_MEuse': Trg_MEuse,
    'An_MPuse_g_Trg': An_MPuse_g_Trg
}

    # Return so they can be viewed in environment
    return diet_info, animal_input, feed_data, equation_selection, AA_values, model_results

# Assign values here so they can be see in environment
diet_info, animal_input, feed_data, equation_selection, AA_values, model_results = NASEM_model()

In [25]:
model_results

{'milk_fat': 3.77725932387044,
 'milk_protein': 2.8979341094222835,
 'Mlk_Prod_comp': 34.41891843122913,
 'Mlk_Prod_MPalow': 25.830659741533808,
 'Mlk_Prod_NEalow': 31.45809407417045,
 'Trg_MEuse': 52.19520139410898,
 'An_MPuse_g_Trg': 1998.2242404723052}

In [26]:
# Display results, temporary
def display_diet_values(df):
    components = ['Fd_CP', 'Fd_RUP_base', 'Fd_NDF', 'Fd_ADF', 'Fd_St', 'Fd_CFat', 'Fd_Ash']
    rows = []

    for component in components:
        percent_diet = round(df.loc['Diet', component + '_%_diet']) #.values[0], 2)
        kg_diet = round(df.loc['Diet', component + '_kg/d'])    #.values[0], 2)
        rows.append([component, percent_diet, kg_diet])

    headers = ['Component', '% DM', 'kg/d']

    table = tabulate(rows, headers=headers, tablefmt='fancy_grid', stralign="center")

    return table

model_data = pd.DataFrame(model_results, index=['Value'])

table1 = display_diet_values(diet_info)
table2 = tabulate(model_data, headers='keys', tablefmt='fancy_grid')

table1_rows = table1.split('\n')
table2_rows = table2.split('\n')
max_rows = max(len(table1_rows), len(table2_rows))
table1_rows += [''] * (max_rows - len(table1_rows))
table2_rows += [''] * (max_rows - len(table2_rows))
combined_rows = [f'{row1}    {row2}' for row1, row2 in zip(table1_rows, table2_rows)]

print('\n'.join(combined_rows))

# VARIABLE UNITS
# milk_fat, milk_protein: % milk
# Mlk_Prod_comp, Mlk_Prod_MPalow, Mlk_Prod_NEalow: kg/d milk (Production predicitons)
# Trg_MEuse: Mcal/d, energy requirement
# An_MPuse_g_Trg: g/d, protein requirement

╒═════════════╤════════╤════════╕    ╒═══════╤════════════╤════════════════╤═════════════════╤═══════════════════╤═══════════════════╤═════════════╤══════════════════╕
│  Component  │   % DM │   kg/d │    │       │   milk_fat │   milk_protein │   Mlk_Prod_comp │   Mlk_Prod_MPalow │   Mlk_Prod_NEalow │   Trg_MEuse │   An_MPuse_g_Trg │
╞═════════════╪════════╪════════╡    ╞═══════╪════════════╪════════════════╪═════════════════╪═══════════════════╪═══════════════════╪═════════════╪══════════════════╡
│    Fd_CP    │     14 │      3 │    │ Value │    3.77726 │        2.89793 │         34.4189 │           25.8307 │           31.4581 │     52.1952 │          1998.22 │
├─────────────┼────────┼────────┤    ╘═══════╧════════════╧════════════════╧═════════════════╧═══════════════════╧═══════════════════╧═════════════╧══════════════════╛
│ Fd_RUP_base │      5 │      1 │    
├─────────────┼────────┼────────┤    
│   Fd_NDF    │     36 │      9 │    
├─────────────┼────────┼────────┤    
│   Fd_A